**Import required libraries**
**Connect to s3**

Load both the csv file from Part 1 **pr.data.0.Current** and the json file from Part 2 as dataframes

In [ ]:
import boto3
import pandas as pd
from io import StringIO
import json
import os
from dotenv import load_dotenv

#Load environment variable from .env file
load_dotenv("<local_path>")
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
BUCKET = os.environ.get("S3_BUCKET") 
BLS_DATA_FILE_NAME = os.environ.get("BLS_DATA_FILE_NAME")
POPULATION_DATA_FILE_NAME = os.environ.get("POPULATION_DATA_FILE_NAME")

s3_client = boto3.client('s3',aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


response = s3_client.get_object(Bucket=BUCKET, Key=BLS_DATA_FILE_NAME)
csv_content = response['Body'].read().decode('utf-8')

j_response = s3_client.get_object(Bucket=BUCKET, Key=POPULATION_DATA_FILE_NAME)
json_content = j_response['Body'].read()


j = json.loads(json_content)

df_bls_csv= pd.read_csv(StringIO(csv_content), delimiter='\t') # Series data from bls as csv
df_population_json = pd.DataFrame(j['data']) # Population data  as Json  
df_bls_csv

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
36940,PRS88003203,2024,Q01,116.936,NaN
36941,PRS88003203,2024,Q02,116.544,NaN
36942,PRS88003203,2024,Q03,116.593,NaN
36943,PRS88003203,2024,Q04,116.622,NaN


Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [ ]:
# Filter data between [2013, 2018] inclusive
population_stats = df_population_json[(df_population_json["Year"].astype(int) >= 2013) &
                              (df_population_json["Year"].astype(int) <= 2018)]
# Display mean and standard deviation of population
mean_population = population_stats["Population"].mean().item()
std_population =  population_stats["Population"].std().item()
print(f"Mean of Population: {mean_population: .2f}") 
print(f"Std Deviation of Population: {std_population: .2f}") 

Mean of Population:  317437383.00
Std Deviation of Population:  4257089.54


Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [32]:
# Remove columns names whitespace
df_bls_csv.rename(columns=lambda x: x.strip(), inplace=True)

# Generate report
max_value_series = df_bls_csv.groupby(["series_id", "year"], as_index=False)["value"].agg("sum")
max_value_series = max_value_series.sort_values("value", ascending=False).drop_duplicates("series_id", keep="first").sort_index().reset_index(drop=True)
max_value_series

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2024,861.350
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset).

In [33]:
specific_series = df_bls_csv.loc[df_bls_csv["series_id"].str.contains("PRS30006032", case=False)]
specific_series = specific_series[specific_series["period"].str.contains("Q01", case=False)]
# Filter population data
population_extract = df_population_json["Population"].astype(int)
df_population_json["Year"] = df_population_json["Year"].astype(int)
# Merge and filter results
result = pd.merge(specific_series, df_population_json, left_on="year", right_on="Year", how="left")
result[["series_id", "year", "period", "value", "Population"]].sort_values("year", ascending=False)

,series_id,year,period,value,Population
30,PRS30006032,2025,Q01,0.5,NaN
29,PRS30006032,2024,Q01,-0.7,NaN
28,PRS30006032,2023,Q01,0.3,332387540.0
27,PRS30006032,2022,Q01,5.3,331097593.0
26,PRS30006032,2021,Q01,0.7,329725481.0
25,PRS30006032,2020,Q01,-7.0,326569308.0
24,PRS30006032,2019,Q01,-1.6,324697795.0
23,PRS30006032,2018,Q01,0.5,322903030.0
22,PRS30006032,2017,Q01,0.9,321004407.0
21,PRS30006032,2016,Q01,-1.4,318558162.0
